In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import ast
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('jointdf.csv', index_col=0) # Taking only common words between title and content

#### finding words with tags having occurence more than 1 

In [3]:
bools = ~(df.iloc[:,1].str.contains(r"'.*': [0-1](,|})?") & (~(df.iloc[:,1].str.contains(r"'.*': [^0-1](,|})?"))))

In [107]:
uni = df[bools].reset_index(drop = True)

In [108]:
# adding the maximum value of keys in a new column
uni["max_val"] = 0

In [109]:
uni.index = uni.Jword
uni = uni.iloc[:,1:3]

In [110]:
# normalizing the values
for i in range(uni.shape[0]):
    f = ast.literal_eval(uni.iloc[i,0])
    key = list(f.keys())
    val = list(f.values())
    max_val = max(val)
    uni.iloc[i,1] = max_val
    try:
        val = list(map(lambda x: x/max_val, val))
    except:
        print(i) # ZeroDivisionError
    d = dict(zip(key,val))
    uni.iloc[i,0] = uni.iloc[i,0].replace(uni.iloc[i,0],str(d))

In [111]:
# a dummy training df 
content = {"Few words":["lucky","eating"],
           "tags":[["Apartments", "Buildings", "NEWTAG"],["Fog", "Sunny", "NEWTAG"]]}
dummy = pd.DataFrame(content)

### Training with AdaBoost

In [113]:
for h in range(dummy.shape[0]):  
    word = dummy.iloc[h,0]              # input word
    vals = dummy.iloc[h,1]              # input tags
    
    # Assuming the word is present in the original look up dictionary
    f = ast.literal_eval(uni.loc[word].Tags) # holding look up dictionary
    
    key = list(f.keys())
    val = list(f.values())

    classes = []
    misclasses = []

    # Accounting for right classified and misclassified tags
    for i in range(len(vals)):

        if vals[i] in key:
            classes.append(vals[i])        

        else:
            misclasses.append(vals[i])       


    right_classified = len(classes)
    wrong_classified = len(misclasses)

    # Updating the weights for rightly classifieds

    epsilon = wrong_classified/(right_classified + wrong_classified)

    for i in range(right_classified):

        # fetching the index of class in orig.
        for j in range(len(key)):
            if classes[i] == key[j]:
                break # tag stored at jth position

        # updating the weight
        val[j] = (1/(0.5*(1-epsilon))) * val[j]


    # Updating the weights for wrongly classifieds
    for i in range(wrong_classified):

        # fetching the index of class in orig.
        flag = 0
        for j in range(len(key)):
            if misclasses[i] == key[j]:
                flag = 1
                break # tag stored at jth position

        # updating the weight

        # CASE 1: TAG is present is original dictionary but not in input dictionary
        if flag == 1:
            val[j] = (1/(0.5*epsilon)) * val[j]
        else:
        # CASE 2: A new TAG appears
            key.append(misclasses[i])
            val.append(1/uni.loc[word].max_val)
    
    d = dict(zip(key,val))
    uni.loc[word,"Tags"] = uni.loc[word].Tags.replace(uni.loc[word].Tags,str(d))

In [ ]:
# Take a test data and use the word to associate the probabilities (in terms of weights)